In [1]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2


# This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
repo_name = 'republic-project'
repo_dir = os.path.split(os.getcwd())[0].split(repo_name)[0] + repo_name
print(repo_dir)
if repo_dir not in sys.path:
    sys.path.append(repo_dir)



/Users/femkegordijn/republic-project


In [2]:
from republic.elastic.republic_elasticsearch import initialize_es
from republic.config.republic_config import set_config_inventory_num

inv_num = 3760
inv_config = set_config_inventory_num(inv_num, ocr_type="pagexml")

rep_es = initialize_es(host_type="external")

rep_es

settings.py is missing a prov_host_url variable. Consider adding a line 'prov_host_url = None' to your settings.py


In [3]:
import republic.extraction.extract_resolution_metadata as extract_res

# initialise a set of searchers that have all the proposition phrases
# 1. a FuzzyPhraseSearcher that has all the known phrases related to propositions
# 2. a FuzzyTemplateSearcher that has some known proposition opening templates that
#    tries to find phrases that together match the template
# 3. a VariableMatcher that identifies the text between phrase matches in the template
#    and labels these in-between texts as part of the proposer or location.
prop_searchers = extract_res.generate_proposition_searchers()

for searcher_type in prop_searchers:
    print(searcher_type, prop_searchers[searcher_type], '\n')

phrase <fuzzy_search.fuzzy_phrase_searcher.FuzzyPhraseSearcher object at 0x7faf827a8fd0> 

template {'proposition_from_correspondence': FuzzyTemplateSearcher(template=FuzzyTemplate(labels={'addressed_to', 'representation_relation', 'person_role', 'proposition_opening', 'residence_relation', 'affairs_relation', 'person_location', 'proposition_verb', 'location_relation', 'proposer_name', 'temporal_reference', 'correspondence_from', 'resolution_relation', 'addressee_name', 'location', 'organisation', 'title', 'person_name_prefix'}, required=['proposition_opening'])), 'proposition_from_statement': FuzzyTemplateSearcher(template=FuzzyTemplate(labels={'addressed_to', 'representation_relation', 'person_role', 'proposition_opening', 'residence_relation', 'affairs_relation', 'proposition_verb', 'location_relation', 'proposer_name', 'resident_in', 'temporal_reference', 'resolution_relation', 'addressee_name', 'location', 'organisation', 'title', 'person_name_prefix'}, required=['proposition_open

In [4]:
# Fetch some resolutions for testing

resolutions = []
for ri, res in enumerate(rep_es.scroll_inventory_resolutions(inv_num)):
    resolutions.append(res)
    if len(resolutions) == 10:
        break

total hits: {'value': 7088, 'relation': 'eq'} 	hits per scroll: 10


In [5]:
import json

# The skip formulas are proposition opening formulas for which we don't know 
# the structure of the full opening sentence, so there is no fitting template.

skip_formulas = {
    'heeft aan haar Hoog Mog. voorgedragen',
    'heeft ter Vergadering gecommuniceert ',
}

for ri, resolution in enumerate(resolutions):
    if len(resolution.evidence) == 0:
        print('resolution without evidence:', resolution.metadata)
    # Fetch all phrase matches for the resolution.
    phrase_matches = extract_res.get_paragraph_phrase_matches(rep_es, resolution)
    # create a copy of the resolution with extended metadata, using the template
    # and variable searchers.
    new_resolution = extract_res.add_resolution_metadata(resolution, phrase_matches,
                                                         prop_searchers['template'],
                                                         prop_searchers['variable'])
    if 'proposition_type' not in new_resolution.metadata or new_resolution.metadata['proposition_type'] is None:
        new_resolution.metadata['proposition_type'] = 'unknown'
    print(json.dumps(new_resolution.metadata, indent=4))


RequestError: RequestError(400, 'parsing_exception', 'no [query] registered for [query]')

In [6]:
import elasticsearch

In [7]:
elasticsearch.__version__


(7, 17, 0)